# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv('../WeatherPy/Output_CSV_PNGs/city_weather.csv')
vacation_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,jackson,42.42,-122.83,72.320,30,1,1.50
1,pangai,-19.80,-174.35,82.400,83,75,9.30
2,balabac,7.99,117.06,80.042,74,71,2.59
3,bayangol,50.70,103.47,28.616,59,0,1.16
4,smolenka,52.14,113.50,21.200,62,58,0.89
...,...,...,...,...,...,...,...
577,san rafael,-13.20,49.05,70.736,84,0,1.63
578,port said,51.03,-4.22,49.802,87,21,1.50
579,lom sak,66.83,70.83,16.178,96,100,7.15
580,gamboula,49.76,15.91,45.698,36,0,2.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Locations
locations = vacation_df[["Latitude", "Longitude"]]
humidity = vacation_df["Humidity (%)"].astype(float)


In [4]:

#create a blank canvas of google maps
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=100,point_radius = 2)

# add layer to the figure
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# arrive at narrowed down cities fitting the conditions
condition1 = vacation_df[vacation_df["Temperature (F)"] > 70]
condition2 = condition1[condition1["Temperature (F)"] < 80]
condition3 = condition2[condition2['Wind Speed (mph)'] < 10]
narrowed_city_df = condition3[condition3['Cloudiness (%)'] == 0] 
narrowed_city_df


,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
104,dikson,-15.20,12.15,75.074,52,0,4.90
188,meyungs,14.90,-24.50,77.288,61,0,6.29
209,ostrovnoy,-20.28,44.28,75.236,68,0,4.28
220,tumannyy,25.21,64.64,75.974,72,0,3.16
221,los llanos de aridane,14.40,46.47,70.088,22,0,1.62
269,sorong,12.48,-16.55,77.036,74,0,2.10
316,slave lake,-12.58,13.41,76.928,78,0,0.45
334,kruisfontein,20.50,-10.07,79.016,14,0,4.14
403,inta,20.58,-75.24,73.454,78,0,2.15
438,charlottetown,11.28,-15.83,79.196,74,0,7.31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#

KeyError: 0

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map